In [1]:
import os, sys
import socket
from random import random
from operator import add
from pyspark.sql import SparkSession

os.environ['PYSPARK_PYTHON'] = 'python3'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3'

sparkSession = SparkSession.builder \
    .master("k8s://https://kubernetes.default.svc.cluster.local:443") \
    .appName("spark-k8s-test") \
    .config("spark.submit.deployMode","client") \
    .config("spark.kubernetes.container.image", "gudari/spark-py:v2.4.3-5") \
    .config("spark.kubernetes.namespace", "jhub") \
    .config("spark.executor.instances", "2") \
    .config("spark.kubernetes.authenticate.oauthTokenFile", "/var/run/secrets/kubernetes.io/serviceaccount/token") \
    .config("spark.kubernetes.authenticate.caCertFile", "/var/run/secrets/kubernetes.io/serviceaccount/ca.crt") \
    .config('spark.driver.host', socket.gethostbyname(socket.gethostname())) \
    .getOrCreate()
sc = sparkSession.sparkContext

In [2]:
partitions = 2
n = 100000 * partitions

def f(_):
    x = random() * 2 - 1
    y = random() * 2 - 1
    return 1 if x ** 2 + y ** 2 <= 1 else 0

count = sc.parallelize(range(1, n + 1), partitions).map(f).reduce(add)
print("Pi is roughly %f" % (4.0 * count / n))

Pi is roughly 3.142840
